# Crop Segmentation and Classification

In this notebook project, we demonstrate and compare techniques for segmentation and classification of crops using Planet imagery. A metric will be established to quantify accuracy of crop classification and will be used to compare technique results.

The notebooks, in order of operations, are:
1. [Identify area of interest and data](identify-datasets.ipynb): Identify and store ground truth data, area of interest, and Planet scene.
2. ..

## Usage Notes

This notebook project was developed in a Docker container. This [Dockerfile](Dockerfile) was used to build the image.